In [1]:
# test to breach the 100 mb limit and preferably get to 6 GB using knowledge base with a custom data source. We will upload all the files manually

In [1]:
import os
import boto3
from config import *
import base64

In [2]:
print(boto3.__version__)

1.37.1


In [3]:
bedrock_agent = boto3.Session().client('bedrock-agent')

In [4]:
path = "/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images"
print(path)

/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images


In [5]:
counter = 0
for filename in os.listdir(path):
    if counter %1000 == 0:
        print(f"{counter}th == {filename}")
    counter +=1

0th == 1000092795.jpg
1000th == 131624221.jpg
2000th == 1572286502.jpg
3000th == 2037264035.jpg
4000th == 2201202388.jpg
5000th == 233242340.jpg
6000th == 2450486758.jpg
7000th == 2573009825.jpg
8000th == 2705102979.jpg
9000th == 2834032170.jpg
10000th == 2954929479.jpg
11000th == 3085592876.jpg
12000th == 3201065069.jpg
13000th == 3315323307.jpg
14000th == 3424424006.jpg
15000th == 3533145793.jpg
16000th == 3643974707.jpg
17000th == 378453580.jpg
18000th == 4011411161.jpg
19000th == 4273698621.jpg
20000th == 4464227525.jpg
21000th == 4575727965.jpg
22000th == 4676717700.jpg
23000th == 4752984291.jpg
24000th == 4830307454.jpg
25000th == 491484504.jpg
26000th == 503357436.jpg
27000th == 5447844412.jpg
28000th == 6041156608.jpg
29000th == 67491882.jpg
30000th == 7283335868.jpg
31000th == 80474519.jpg


In [6]:
# If you're using PIL/Pillow Image object
from PIL import Image
import io
import base64

In [7]:
# If you have an image file path
def image_to_bytes(image_path):
    with open(image_path, 'rb') as image:
        return {'data': base64.b64encode(image.read())}

def pil_image_to_bytes(image_path):
    # Open the image file
    pil_image = Image.open(image_path)
    # Convert PIL Image to bytes
    img_byte_arr = io.BytesIO()
    pil_image.save(img_byte_arr, format=pil_image.format or 'JPEG')
    img_byte_arr = img_byte_arr.getvalue()
    return {'data': base64.b64encode(img_byte_arr)}

In [8]:
## example usage
image_path = path + '/1314231418.jpg'
image_bytes = pil_image_to_bytes(image_path)
image_bytes_data = image_bytes['data']

In [9]:
print(type(image_bytes['data']))

<class 'bytes'>


In [10]:
import random
import string
import time

In [11]:
def generate_random_code():
    # Generate 4 random characters (uppercase letters)
    characters = ''.join(random.choices(string.ascii_uppercase, k=4))
    
    # Generate 12 digit number combining timestamp and random numbers
    timestamp = str(int(time.time()))
    random_digits = ''.join(random.choices(string.digits, k=6))
    number = (timestamp + random_digits)[-12:]  # Ensure 12 digits
    
    return characters, number

In [12]:
# now we will test uploading a single image
def upload_image(image_path):
    image_bytes = pil_image_to_bytes(image_path)
    image_bytes_data = image_bytes['data']
    
    random_code = generate_random_code()
    documentIdentifier = str(random_code[0]) + str(random_code[1]) 
    document = {
        'content': {
            'custom': {
                'customDocumentIdentifier': {
                    'id': documentIdentifier
                },
                'inlineContent': {
                    'byteContent': {
                        'data': image_bytes_data,
                        'mimeType': 'image/jpeg'
                    },
                    'type': 'BYTE'
                },
                'sourceType': 'IN_LINE'
            },
            'dataSourceType': 'CUSTOM',
        }
    }
    documents = [document]

    response = bedrock_agent.ingest_knowledge_base_documents(
        dataSourceId = data_source_id, documents = documents, knowledgeBaseId = knowledgebase_id
    )
    return response

In [13]:
def get_data_source():
    data_source = bedrock_agent.get_data_source(
        dataSourceId = data_source_id,
        knowledgeBaseId =knowledgebase_id
    )
    return data_source

In [18]:
counter = 0
for filename in os.listdir(path):
    if counter == 10:
        break
    image_path = path + "/" + filename
    upload_response = upload_image(image_path)
    timer = 0
    print(" --- Updload response")
    print(" --- ")
    while timer < 10:
        data_source = get_data_source()
        time.sleep(30)
        timer +=1
    print(image_path)
    counter +=1

 --- Updload response
 --- 
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000092795.jpg
 --- Updload response
 --- 
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/10002456.jpg
 --- Updload response
 --- 
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000268201.jpg
 --- Updload response
 --- 
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000344755.jpg
 --- Updload response
 --- 
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000366164.jpg
 --- Updload response
 --- 
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000523639.jpg


**according to** [documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/ingest_knowledge_base_documents.html) uploading images using the API is not supported. see:

```
mimeType (string) – [REQUIRED]
The MIME type of the content. For a list of MIME types, see Media Types. The following MIME types are supported:
text/plain
text/html
text/csv
text/vtt
message/rfc822
application/xhtml+xml
application/pdf
application/msword
application/vnd.ms-word.document.macroenabled.12
application/vnd.ms-word.template.macroenabled.12
application/vnd.ms-excel
application/vnd.ms-excel.addin.macroenabled.12
application/vnd.ms-excel.sheet.macroenabled.12
application/vnd.ms-excel.template.macroenabled.12
application/vnd.ms-excel.sheet.binary.macroenabled.12
application/vnd.ms-spreadsheetml
application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
application/vnd.openxmlformats-officedocument.spreadsheetml.template
application/vnd.openxmlformats-officedocument.wordprocessingml.document
application/vnd.openxmlformats-officedocument.wordprocessingml.template
```
and I am getting logs of this kind:
```
{
    "event_timestamp": 1740509325142,
    "event": {
        "document_location": {
            "customDocument_location": {
                "id": "***"
            },
            "type": "CUSTOM"
        },
        "request_id": "********",
        "data_source_id": "********",
        "status_reasons": [
            "Resource not processed due to a service exception. Please reach out to Amazon Bedrock technical support team for further assistance, or contact technical support at aws.amazon.com/contact-us/."
        ],
        "knowledge_base_arn": "arn:aws:bedrock:us-east-1:**********:knowledge-base/********",
        "status": "RESOURCE_IGNORED"
    },
    "event_version": "1.0",
    "event_type": "IngestKnowledgeBaseDocuments.ResourceStatusChanged",
    "level": "WARN"
}
```

uploading 6 GB of data will take an excesive amount of time and resources to test. For instance, when I first attempted to upload a 17 MB pdf file (the bedrock user guide actually) which contains more than 2500 pages, It was taking longer than 8 hours and I had to stop the test for that reason. Hence uploading 6 GB of data will take 118 days.

In [14]:
s3_location = 's3://mainbucketrockhight5461/test/knowledge-bases/direct_KB_upload/1image/371897.jpg'

In [15]:
# I will attempt to upload the images using s3 direct upload to custom source
def s3_upload_image(s3_location):
    
    random_code = generate_random_code()
    documentIdentifier = str(random_code[0]) + str(random_code[1]) 
    document = { 
        'content':{
            'dataSourceType': 'CUSTOM',
            'custom':{
                'customDocumentIdentifier': {
                    'id': documentIdentifier
                },
                's3Location': {
                    'bucketOwnerAccountId': bucket_owner_account,
                    'uri': s3_location
                },
                'sourceType': 'S3_LOCATION'
            }
        }
    }
    
    documents = [document]

    response = bedrock_agent.ingest_knowledge_base_documents(
        dataSourceId = data_source_id, documents = documents, knowledgeBaseId = knowledgebase_id
    )
    return response

In [28]:
s3_upload_response = s3_upload_image(s3_location)

In [29]:
print(s3_upload_response)

{'ResponseMetadata': {'RequestId': '04ac8cca-2587-49ea-9a00-6e8c15ea4814', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Wed, 26 Feb 2025 04:02:18 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': '04ac8cca-2587-49ea-9a00-6e8c15ea4814', 'x-amz-apigw-id': 'GkvLYG8WIAMEZjg=', 'x-amzn-trace-id': 'Root=1-67be9248-16c53ad15c2c7c7827578517'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'E3NRB1GDE0', 'identifier': {'custom': {'id': 'GUAT542536187863'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '5WYB0DMRMN', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 2, 26, 4, 2, 17, 795270, tzinfo=tzlocal())}]}


In [30]:
# the request failed, I will test now using a json file
s3_response = s3_upload_image('s3://mainbucketrockhight5461/test/documentation/bedrock-user-guide.pdf.metadata.json')

In [31]:
print(s3_response)

{'ResponseMetadata': {'RequestId': '3d353762-49c1-460a-935e-c411d936acb9', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Wed, 26 Feb 2025 04:06:11 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': '3d353762-49c1-460a-935e-c411d936acb9', 'x-amz-apigw-id': 'Gkvv0FZ1IAMED8A=', 'x-amzn-trace-id': 'Root=1-67be9331-12e06a2f070686eb35bd5087'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'E3NRB1GDE0', 'identifier': {'custom': {'id': 'QDUB542769316690'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '5WYB0DMRMN', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 2, 26, 4, 6, 10, 894650, tzinfo=tzlocal())}]}


In [ ]:
## I think the ingestions might be failing because we already reached the limit actually. So I will attempt with a new knowledge base

In [19]:
# I will attempt to upload the images using s3 direct upload to custom source
def s3_upload_image_nds(s3_location, data_source_id):
    
    random_code = generate_random_code()
    documentIdentifier = str(random_code[0]) + str(random_code[1]) 
    document = { 
        'content':{
            'dataSourceType': 'CUSTOM',
            'custom':{
                'customDocumentIdentifier': {
                    'id': documentIdentifier
                },
                's3Location': {
                    'bucketOwnerAccountId': bucket_owner_account,
                    'uri': s3_location
                },
                'sourceType': 'S3_LOCATION'
            }
        }
    }
    
    documents = [document]

    response = bedrock_agent.ingest_knowledge_base_documents(
        dataSourceId = data_source_id, documents = documents, knowledgeBaseId = knowledgebase_id
    )
    return response

# now we will test uploading a single image
def upload_image_nds(image_path, data_source_id):
    image_bytes = pil_image_to_bytes(image_path)
    image_bytes_data = image_bytes['data']
    
    random_code = generate_random_code()
    documentIdentifier = str(random_code[0]) + str(random_code[1]) 
    document = {
        'content': {
            'custom': {
                'customDocumentIdentifier': {
                    'id': documentIdentifier
                },
                'inlineContent': {
                    'byteContent': {
                        'data': image_bytes_data,
                        'mimeType': 'image/jpeg'
                    },
                    'type': 'BYTE'
                },
                'sourceType': 'IN_LINE'
            },
            'dataSourceType': 'CUSTOM',
        }
    }
    documents = [document]

    response = bedrock_agent.ingest_knowledge_base_documents(
        dataSourceId = data_source_id, documents = documents, knowledgeBaseId = knowledgebase_id
    )
    return response

In [22]:
counter = 0
for filename in os.listdir(path):
    if counter == 10:
        break
    image_path = path + "/" + filename
    upload_response = upload_image_nds(image_path, new_data_source)
    print(upload_response)
    timer = 0
    time.sleep(5)
    print(image_path)
    counter +=1

{'ResponseMetadata': {'RequestId': 'afe0ff9f-8c2d-4304-aea5-0a8e80b0bcbe', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Wed, 26 Feb 2025 15:46:04 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': 'afe0ff9f-8c2d-4304-aea5-0a8e80b0bcbe', 'x-amz-apigw-id': 'GmWRMGhvoAMEpxA=', 'x-amzn-trace-id': 'Root=1-67bf373a-42006299422ae94d04dcb014'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'G7YKZ4SMPY', 'identifier': {'custom': {'id': 'SAKJ584762960443'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '5WYB0DMRMN', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 2, 26, 15, 46, 4, 14441, tzinfo=tzlocal())}]}
/home/sagemaker-user/.cache/kagglehub/datasets/hsankesara/flickr-image-dataset/versions/1/flickr30k_images/flickr30k_images/1000092795.jpg
{'ResponseMetadata': {'RequestId': 'bc04d6e4-42e6-4b05-9b1d-ca69ac02f65e', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Wed, 26 Feb 2025 15:46:11 GMT', 'content

In [23]:
# uploads are failing manually apparently so I will attempt to upload the image using s3 source

In [28]:
s3_response = s3_upload_image_nds('s3://mainbucketrockhight5461/test/knowledge-bases/documentacion.txt', new_data_source)

In [29]:
print(s3_response)

{'ResponseMetadata': {'RequestId': '17d28391-4558-471f-9dca-a392434df23d', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Wed, 26 Feb 2025 15:51:06 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': '17d28391-4558-471f-9dca-a392434df23d', 'x-amz-apigw-id': 'GmXAZHGvoAMENeQ=', 'x-amzn-trace-id': 'Root=1-67bf3868-1435b7b73058e3901ae518c3'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'G7YKZ4SMPY', 'identifier': {'custom': {'id': 'CYIA585064842622'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '5WYB0DMRMN', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 2, 26, 15, 51, 6, 198482, tzinfo=tzlocal())}]}


In [33]:
# now we will attempt to upload txt files and json files inline

def text_to_bytes(file_path, encoding='utf-8'):
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content.encode(encoding)
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

def upload_txt_custom(text_path, data_source_id, mime_type):
    bytes_data = text_to_bytes(text_path)
    
    random_code = generate_random_code()
    documentIdentifier = str(random_code[0]) + str(random_code[1]) 
    document = {
        'content': {
            'custom': {
                'customDocumentIdentifier': {
                    'id': documentIdentifier
                },
                'inlineContent': {
                    'byteContent': {
                        'data': bytes_data,
                        'mimeType': mime_type
                    },
                    'type': 'BYTE'
                },
                'sourceType': 'IN_LINE'
            },
            'dataSourceType': 'CUSTOM',
        }
    }
    documents = [document]

    response = bedrock_agent.ingest_knowledge_base_documents(
        dataSourceId = data_source_id, documents = documents, knowledgeBaseId = knowledgebase_id
    )
    return response

In [34]:
mime_type = "text/plain"

In [37]:
file = "/home/sagemaker-user/user-default-efs/documents/mygithub/aws-samples/bedrock/miscelanious/test.txt"

In [39]:
file_response = upload_txt_custom(file, new_data_source, mime_type)

In [40]:
print(file_response)

{'ResponseMetadata': {'RequestId': '7e1b8558-fe10-4b13-802f-fcc5f66eaeea', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Wed, 26 Feb 2025 16:07:08 GMT', 'content-type': 'application/json', 'content-length': '223', 'connection': 'keep-alive', 'x-amzn-requestid': '7e1b8558-fe10-4b13-802f-fcc5f66eaeea', 'x-amz-apigw-id': 'GmZWuEhHoAMEsqg=', 'x-amzn-trace-id': 'Root=1-67bf3c2a-6522f2945fc8998472b8d1fc'}, 'RetryAttempts': 0}, 'documentDetails': [{'dataSourceId': 'G7YKZ4SMPY', 'identifier': {'custom': {'id': 'XXDL586026923417'}, 'dataSourceType': 'CUSTOM'}, 'knowledgeBaseId': '5WYB0DMRMN', 'status': 'STARTING', 'updatedAt': datetime.datetime(2025, 2, 26, 16, 7, 8, 532765, tzinfo=tzlocal())}]}
